In [ ]:
# Youtube API reference
https://developers.google.com/youtube/v3/docs/commentThreads/list
https://developers.google.com/youtube/v3/docs/errors

In [1]:
!pip install emoji --upgrade

     |████████████████████████████████| 170 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=7580da2b34841f37b76f5d19350f1d3e2bd4e2db8c5a9448ac158646844b3097
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [2]:
import urllib.request
import re

from googleapiclient.discovery import build
import configparser
import os
from emoji import demojize
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
html = urllib.request.urlopen("https://www.youtube.com/results?search_query=COVID")
# print(html.read().decode())

In [5]:
video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
print(video_ids)

['ih4xVzLQP24', '4yfPs6pUjQI', 'DqSpzm2jsgw', 'N4vfuuReV5c', 'UdQri8yLXFA', 's_3jKDRnF4c', 'ewFkYdEv2w0', '1z_mBM3OWMM', 'w2KlawRXjGM', 'Ug9xQ7LQnsw', 'DqSpzm2jsgw', 'A-1KmtpvM9M', 'ZL1z3Uju-I0', 'DCdxsnRF1Fk', 'yH526AyO6IM', 'MiqRBaEehIU', '08pHlsFYfBs', 'b7pAc7-vTvo', 'fudTj7i4gJA', 'fudTj7i4gJA', 'fUcAhhqwGVw', 'BSQrskqqIKE', 'T69-aH3s0vw', 'e8FCiP2IXWM', 'k9wRPOeUhls', 'LKT7fGL_Uh0', '5DGwOJXSxqg', 'BtN-goy9VOY', '5LuOJQFGS1s', 'U8r3oTVMtQ0', 'zkVsXOZguLg', '0b_eHBZLM6U', 'PkUlCIFE45M', 'SeQ7CDkr9SY', 'VgBiPyirG-A', '3OXJlRrXmQc', '8KwHceAtQP8', 'RcBk0n1YjQY', 'UdQri8yLXFA', 'l28OXXl35SA', 'hwX5D4PP9jo', 'AqI4eDZFQRA', 'jUDL7VHN5yE', 'BtELl7-zE_w', 'HxDXKB59JFg', 'Ory4ut0SDso', 's7In-KGxduU', 'GOimxFhJ4Pk', 'ZfFe4RngT-w', '1z_mBM3OWMM', 'OF9ddutmfOI', 'GLLYQdyLlyI', 'Ec3FDyYPsms', '-DBR_2BJ0ko', '8wTi2o_pCWQ', 'U0d2zmr0WGo', 'w0Tw73Ovf3M', 'kwPF2GsW8bg', 'L2k46LDp2ac']


In [6]:
len(video_ids)

59

In [7]:
config = configparser.ConfigParser()
config.read_file(open('YoutubeAPI.cfg'))

os.environ["Youtube_API_Access_KEY"]= config['Youtube API']['API_KEY']

youtube = build('youtube', 'v3', developerKey = os.environ["Youtube_API_Access_KEY"] )

In [15]:
Comment_list = []

def extract_comments_eh(V_ID):

    request = youtube.commentThreads().list(
            part="id, snippet, replies",
            order="time",
            # order="relevance",
            videoId=V_ID,
            maxResults = 20
        )
    
    try:
        response = request.execute()

        for i in range(len(response['items'])):
            Comment_list.append(demojize(response['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal']).replace("\n", ";").replace("-", "").replace("      ", ""))


        while "nextPageToken" in response:
            nextPage = response['nextPageToken']

            request = youtube.commentThreads().list(
                part="id, snippet, replies",
                order="time",
                videoId=V_ID,
                pageToken=nextPage,
                maxResults = 20
            )

            response = request.execute()

            for i in range(len(response['items'])):
                Comment_list.append(demojize(response['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal']).replace("\n", ";").replace("-", "").replace("      ", ""))
            if "nextPageToken" not in response:
                break

        df = pd.DataFrame(Comment_list, columns=['Youtube Comments'])
        df['Video ID'] = V_ID

        return(df)

    except Exception:
        print("Video with ID " + V_ID + " skipped due to no comments allowed")

In [31]:
for i in range(len(video_ids)):
    print('This is iteration', i )
    globals()[f"DF_{i}"] = extract_comments_eh(video_ids[i])

This is iteration 0
This is iteration 1
This is iteration 2
This is iteration 3
This is iteration 4
This is iteration 5
This is iteration 6
This is iteration 7
This is iteration 8
This is iteration 9
This is iteration 10
This is iteration 11
This is iteration 12
This is iteration 13
This is iteration 14
Video with ID yH526AyO6IM skipped due to no comments allowed
This is iteration 15
This is iteration 16
Video with ID 08pHlsFYfBs skipped due to no comments allowed
This is iteration 17
This is iteration 18
This is iteration 19
This is iteration 20
This is iteration 21
This is iteration 22
This is iteration 23
This is iteration 24
This is iteration 25
This is iteration 26
This is iteration 27
This is iteration 28
Video with ID 5LuOJQFGS1s skipped due to no comments allowed
This is iteration 29
Video with ID U8r3oTVMtQ0 skipped due to no comments allowed
This is iteration 30
Video with ID zkVsXOZguLg skipped due to no comments allowed
This is iteration 31
Video with ID 0b_eHBZLM6U skipped

In [32]:
frames = [DF_0,	DF_1,	DF_2,	DF_3,	DF_4,	DF_5,	DF_6,	DF_7,	DF_8,	DF_9,	DF_10, DF_11,	DF_12, DF_13, DF_15, DF_17,	DF_18, DF_19,\
          DF_20, DF_21,	DF_22, DF_23,	DF_24, DF_25,	DF_26, DF_27,	DF_28, DF_44, DF_48 ]

result = pd.concat(frames)

In [33]:
result.count()

Youtube Comments    355147
Video ID            355147
dtype: int64

In [34]:
result[1000:1005]

,Youtube Comments,Video ID
1000,Because the government is only interested in controlling the people. And if the government demanded a working vaccine. Then their power and control would fade as the virus does. Control not health is what democrats want.,ih4xVzLQP24
1001,14 of my family members (all fully vaccinated) got covid from our thanksgiving,ih4xVzLQP24
1002,Because it’s NOT a vaccine,ih4xVzLQP24
1003,So the point of pushing the vaccine isssss? What? Make it make sense …. It doesn’t protect from Covid so what is the agenda here?,ih4xVzLQP24
1004,Cuz vaccines don't work....they never did and never will.....,ih4xVzLQP24


In [35]:
result[10000:10005]

,Youtube Comments,Video ID
2800,To all my unvaxxed brethren out there: HOLD THE LINE!,UdQri8yLXFA
2801,Because it’s not a vaccine.,UdQri8yLXFA
2802,Because they were so dumb they actually believed in this fake vax,UdQri8yLXFA
2803,Less common than you think that they are but more common than we thought it would be. Wait flu shot to battle wuflu?? Isn't that medical misinformation?,UdQri8yLXFA
2804,I'm fully vaccinated and have covid right now. Fkn sucks! Took vaccine for nothing!,UdQri8yLXFA


In [36]:
result.to_csv('Youtube_Comments_Combined_16Dec_All' + '.csv', encoding = 'utf-8', index=False)